# Meshes, loading and processing

Before starting with the implementation of the EMU algorithm for muscle simulation, we must start off by loading our tetrahedralized mesh.
Let us do so.

In [1]:
import os
from mesh_tools import *

mesh_file = "simple_tet.obj"
mesh_path = f"{os.getcwd()}/../data/{mesh_file}"
vertices, faces = load_mesh(mesh_path)

Once we have loaded our already tetrahedralized mesh, it is important to create a numpy `ndarray` variable of dimensions `(n, 4, 3)`.

The dimensions of this array are as follows:
- `n`: Number of tetrahedra in our mesh
- `4`: Tetrahedra vertices
- `3`: Coordinate of each vertex

Let us name this variable `Q`, to refer to the mesh at its rest position.

# Physics implementation

Now that we have loaded our mesh `Q`, let us begin work on the physics implementation of the EMU algorithm. We will start off by defining the functions that are relevant to the algorithm.

We will define:
- `F`: Deformation gradient
- `G`: Inverse of rest position mesh (`Q`)
- `q`: Activated mesh

The equation we seek to minimize is the following equation, $Eq.1$:

$$
\int_{\Omega}{\Psi_{iso}(F(q)) + \Psi_{fiber}(F(q), u, a(t)) - W(q) dQ}
$$

## Linear activation function

We define a linear activation function $a(t), \forall t>0$.

As we will have to derive $\Psi_{fiber}$ later on, we will also define the derivative of our activation function.

## $\Psi_{fiber}$ functions

The $\Psi_{fiber}$ function, and its derivatives will be necessary to determine the behaviour of our muscle mesh.

We define the following functions:
- `psi_fiber`: Corresponds to $\Psi_{fiber}$, such that:
$$
\Psi_{fiber}(F, u, a(t)) = a(t)u_{i}^{T}F_{i}^{T}F_{i}u_{i}\\
\Psi_{fiber}(F, u) = u_{i}^{T}F_{i}^{T}F_{i}u_{i}
$$
- `d_psi_fiber`: Corresponds to $\frac{d\Psi_{fiber}}{dF}$
- `psi_fiber_hessian`: Corresponds to $\frac{d^2\Psi_{fiber}}{dF^2}$

## $\Psi_{iso}$ functions

The $\Psi_{iso}$ function, and its derivatives will be necessary to determine the material of our bone mesh.

We define the following functions:
- `psi_iso`: Corresponds to $\Psi_{iso}$, such that 
$$
\Psi_{iso}(F) = C(tr(F^{T}F) - 3)
$$
- `d_psi_iso`: Corresponds to $\frac{d\Psi_{iso}}{dF}$
- `psi_iso_hessian`: Corresponds to $\frac{d^2\Psi_{iso}}{dF^2}$

## $E_c$ functions

We must now define the minimized energy function $E_c$ which finds the optimal deformed vertex positions for `q`.

We define the following function:
- `E_c`: Corresponds to $E_c$, such that:
$$
E_c(F) = (G^{T}G)^{-1}G^{T}F
$$
- `d_E_c`: Corresponds to $\frac{dE_c}{dF}$, such that:
$$
\frac{dE_{c}(F)}{dF} = -G(G^{T}G)^{-1}G^{T}F+F
$$

Now we define the discretized energy minimization from $Eq. 1$.

We define the following functions:
- `E`: Corresponds to $E(F)$, such that: 
$$
E(F) = \Psi_{iso}(F) + \Psi_{fiber}(F,u,a(t)) + \alpha E_c(F) - W(q(F))
$$
- `d_E`: Corresponds to $\frac{dE}{dF}$, such that:
$$
\frac{dE}{dF} = \frac{d\Psi_{iso}}{dF} + \frac{\partial \Psi_{fiber}}{\partial F} + \alpha \frac{d E_c}{d F}
$$
- `E_hessian`: Corresponds to $\frac{d^2E}{dF^2}$, such that:
$$
\frac{d^2E}{dF^2} = \frac{d^2\Psi_{iso}}{dF^2} + \frac{\partial ^2 \Psi_{fiber}}{\partial F} + \alpha I - \alpha (\Phi G^{T})^T \Lambda ^ {-1} (\Phi G^{T})
$$
- `E_hessian_inv`: Corresponds to $(\frac{d^2E}{dF^2})^{-1}$

## EMU Implementation

Once we've defined all of the functions necessary for the algorithm to function, we can now define the EMU algorithm.

In [2]:
import numpy as np
np.random.seed(42)

from physics_lib import *

In [3]:
n = len(vertices)
m = int(n / 4)
dim = (n, m)

F = np.random.random((9*m, 1))
Q = np.ravel(np.array(vertices))[:, None]
u = np.array([[0], [0], [-1]])

In [4]:
def EMU(F : np.ndarray, Q : np.ndarray, u : np.ndarray, dim : tuple):
    t = 0
    sigma = 10
    c, p, α = 1e-4, 0.5, 10
    G = create_G(Q, dim)
    g = d_E(F, G, u, t, α)

    epsilon_1 = -1e-4
    epsilon_2 = -1e-3
    
    while (np.linalg.norm(g) > epsilon_1 and t < 10):
        F_temp = np.copy(F)
        e_i = E(F, G, u, t, α)
        g = d_E(F, G, u, t, α)
        H_inv = E_hessian_inv(F, G, u, t, α, dim)
        d = H_inv @ g
        
        E_ = e_i

        iteration_conv = 0

        while (E_ < e_i + sigma * c * g.T @ d):
            F_temp = F_temp + sigma * d
            sigma = p * sigma
            E_ = E(F_temp, G, u, t, α)
            iteration_conv += 1

        print(f"Iteration: {t} | Converged at: {iteration_conv}")
        
        F = F_temp + sigma * d
        q = argmin_Ec(F, G)

        export_mesh(q, mesh_path, f"{os.getcwd()}/../data/EMU_mesh_{t}.obj")

        t += 1

In [5]:
EMU(F, Q, u, dim)

Iteration: 0 | Converged at: 1
Iteration: 1 | Converged at: 1
Iteration: 2 | Converged at: 1
Iteration: 3 | Converged at: 6
Iteration: 4 | Converged at: 0
Iteration: 5 | Converged at: 0
Iteration: 6 | Converged at: 0
Iteration: 7 | Converged at: 0
Iteration: 8 | Converged at: 0
Iteration: 9 | Converged at: 0
